# Tutorial 4 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Setup Guide

## Step 1: Install AgentJo

In [1]:
# !pip install agentjo

## Step 2: Import required functions and setup relevant API keys for your LLM

In [2]:
# Set up API key and do the necessary imports
from agentjo import *
import os

from dotenv import load_dotenv
load_dotenv()

True

## Step 3: Define your own LLM
- Take in a `system_prompt`, `user_prompt`, and outputs llm response string

In [3]:
def llm(system_prompt: str, user_prompt: str) -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI
    
    # define your own LLM here
    client = OpenAI()
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        temperature = 0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Verify that llm function is working
llm(system_prompt = 'You are a classifier to classify the sentiment of a sentence', 
    user_prompt = 'It is a hot and sunny day')

'Neutral'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                 llm = llm)

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decision on which items to include in menu.', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 1, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([dish_price])
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in menu format.')
```

In [5]:
def random_number_from_string(input_string) -> int:
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [6]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price of each dish '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output

In [7]:
dish_price(['spaghetti', 'tomato'])

{'spaghetti': '$8', 'tomato': '$1'}

In [8]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.',
                 llm = llm)

In [9]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.', llm = llm),
    Agent('Boss', 'Makes decisions on which items to include in Menu', llm = llm),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', 
          max_subtasks = 1, llm = llm),
    Agent('Economist', 'Takes in dish names and comes up with pricing for each of them', llm = llm).assign_functions([dish_price])
    ]

In [10]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [11]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

Observation: No subtasks have been completed yet for the assigned task of generating 5 menu items based on Italian food choices.

Thoughts: To complete the assigned task, I need to first generate interesting dish names and descriptions for the Italian menu items. After that, I can gather the ingredients for each dish and finally determine the pricing.

Subtask identified: Generate interesting dish names and descriptions for 5 Italian menu items.

Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for 5 Italian menu items.\n'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for the assigned task of generating interesting dish names and descriptions for 5 Italian menu items.

Thoughts: To complete the remainder of the assigned task, I need to brainstorm creative and appealing dish names along with their descriptions that reflect the essence of Italian cuisine.

Subtask identifi

In [12]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate interesting dish names and descriptions for 5 Italian menu items.
")
{'Agent Output': '1. **Sorrento Sunset Risotto**: A creamy arborio rice dish infused with the vibrant flavors of sun-dried tomatoes, fresh basil, and a hint of lemon zest, evoking the warm hues of a sunset over the Amalfi Coast.\n\n2. **Truffle Whisper Tagliatelle**: Handcrafted tagliatelle pasta tossed in a luxurious truffle cream sauce, garnished with shaved black truffles and a sprinkle of Parmigiano-Reggiano, offering an indulgent taste of Italian elegance.\n\n3. **Vene

In [13]:
my_agent.thoughts

[{'Observation': 'No subtasks have been completed yet for the assigned task of generating 5 menu items based on Italian food choices.\n',
  'Thoughts': 'To complete the assigned task, I need to first generate interesting dish names and descriptions for the Italian menu items. After that, I can gather the ingredients for each dish and finally determine the pricing.\n',
  'Current Subtask': 'Generate interesting dish names and descriptions for 5 Italian menu items.\n',
  'Equipped Function Name': 'Creative Writer',
  'Equipped Function Inputs': {'instruction': 'Generate interesting dish names and descriptions for 5 Italian menu items.\n'}},
 {'Observation': 'The Creative Writer has generated 5 Italian menu items with names and descriptions. The items are:\n1. Sorrento Sunset Risotto\n2. Truffle Whisper Tagliatelle\n3. Venetian Carnival Pizza\n4. Tuscan Herb Frittata\n5. Caprese Symphony Salad\n',
  'Thoughts': 'To complete the remainder of the Assigned Task, I need to generate the ingred

In [14]:
my_agent.reply_user()

'- Name: Sorrento Sunset Risotto\n  Description: A creamy arborio rice dish infused with the vibrant flavors of sun-dried tomatoes, fresh basil, and a hint of lemon zest, evoking the warm hues of a sunset over the Amalfi Coast.\n  Ingredients:\n    - 1 cup Arborio rice\n    - 4 cups vegetable broth\n    - 1/2 cup sun-dried tomatoes, chopped\n    - 1/2 cup fresh basil, chopped\n    - 1 small onion, finely chopped\n    - 2 cloves garlic, minced\n    - 1/2 cup dry white wine\n    - 1/2 cup grated Parmesan cheese\n    - 2 tablespoons olive oil\n    - Zest of 1 lemon\n    - Salt and pepper to taste\n  Pricing: $7\n\n- Name: Truffle Whisper Tagliatelle\n  Description: Handcrafted tagliatelle pasta tossed in a luxurious truffle cream sauce, garnished with shaved black truffles and a sprinkle of Parmigiano-Reggiano, offering an indulgent taste of Italian elegance.\n  Ingredients:\n    - 250g tagliatelle pasta\n    - 1 cup heavy cream\n    - 1/2 cup grated Parmigiano-Reggiano cheese\n    - 1/4 

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [15]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [16]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.')

In [17]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate only 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate only 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Observation: No subtasks have been completed yet for the task of generating 5 Italian dish names and descriptions.

Thoughts: To complete the remainder of the Assigned Task, I need to generate 5 unique and interesting Italian dish names along with their descriptions. I can utilize the equipped function that is designed for general tasks to help create these dish names and descriptions.

Subtask identified: Generate 5 Italian dish names and descriptions using the equipped function.

Getting LLM to perform the following task: Generate 5 Italian dish names and descriptions using the equipped function.

> 1. **Truffle Risotto Delight**: A creamy Arborio rice dish infused with the earthy aroma of black truffles, finished with a sprinkle of Parmigiano-Reggiano and a drizzle of truffle oil for an indulgent experience.

2. **Saff

In [18]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. **Truffle Risotto Delight**: A creamy Arborio rice dish infused with the earthy aroma of black truffles, finished with a sprinkle of Parmigiano-Reggiano and a drizzle of truffle oil for an indulgent experience.\n\n2. **Saffron-Infused Seafood Linguine**: Fresh linguine tossed with a medley of succulent shrimp, mussels, and calamari, all enveloped in a luxurious saffron cream sauce that captures the essence of the Mediterranean.\n\n3. **Pesto Genovese Pizza**: A thin-crust pizz

In [19]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Observation: The Creative Writer has generated 5 Italian dish names and descriptions, which are ready for ingredient generation.

Thoughts: To complete the Assigned Task, I need to generate the ingredients for each of the 5 Italian dishes provided in the output. I will focus on one dish at a time to ensure accuracy and completeness.

Subtask identified: Generate ingredients for the following dishes:
1. Truffle Risotto Delight
2. Saffron-Infused Seafood Linguine
3. Pesto Genovese Pizza
4. Eggplant Parmigiana Stack
5. Limoncello Tiramisu

Getting LLM to perform the following task: Generate ingredients for the following dishes:
1. Truffle Risotto Delight
2. Saffron-Infused Seafood Linguine
3. Pesto Genovese Pizza
4. Eggplant Parmigiana Stack
5. Limoncello Tiramisu

> 1. **Truffle Risotto Delight**:
   - Arborio rice
   - Black truffles
   - Chicken or 

In [20]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Observation: The task is to generate prices for 5 Italian dishes: Truffle Risotto Delight, Saffron-Infused Seafood Linguine, Pesto Genovese Pizza, Eggplant Parmigiana Stack, and Limoncello Tiramisu. 
No previous subtasks have been completed for this task, so I need to start from scratch.

Thoughts: To complete the task, I will use the equipped function that takes in a list of dish names and outputs the price for each dish. 
I need to ensure that I provide the correct list of dish names to the function.

Subtask identified: Use the dish_price function to generate prices for the following dishes:
- Truffle Risotto Delight
- Saffron-Infused Seafood Linguine
- Pesto Genovese Pizza
- Eggplant Parmigiana Stack
- Limoncello Tiramisu

Calling function dish_price with parameters {'list_of_dish_names': ['Truffle Risotto Delight', 'Saffron-Infused Seafood

In [21]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Observation: The completed subtasks have generated a list of 5 Italian dishes along with their descriptions, ingredients, and prices. The dishes are Truffle Risotto Delight, Saffron-Infused Seafood Linguine, Pesto Genovese Pizza, Eggplant Parmigiana Stack, and Limoncello Tiramisu. 

Thoughts: To please Gordon Ramsay, I should select dishes that are not only flavorful but also visually appealing and well-balanced. The Truffle Risotto Delight and Saffron-Infused Seafood Linguine stand out for their luxurious ingredients, while the Pesto Genovese Pizza offers a classic Italian flavor. I need to choose 3 dishes that complement each other and showcase a variety of textures and tastes.

Subtask identified: Select 3 Italian dishes from the list that will appeal to Gordon Ramsay, ensuring a balance of flavors and presentation.

Gettin

In [22]:
# Get a targeted response from your agent, and add this to subtask_completed
my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

'Truffle Risotto Delight, A creamy Arborio rice dish infused with the earthy aroma of black truffles, finished with a sprinkle of Parmigiano-Reggiano and a drizzle of truffle oil for an indulgent experience., \n- Arborio rice\n- Black truffles\n- Chicken or vegetable broth\n- Onion\n- Garlic\n- White wine\n- Parmigiano-Reggiano cheese\n- Truffle oil\n- Fresh parsley, \n$8\n\nSaffron-Infused Seafood Linguine, Fresh linguine tossed with a medley of succulent shrimp, mussels, and calamari, all enveloped in a luxurious saffron cream sauce that captures the essence of the Mediterranean., \n- Linguine pasta\n- Shrimp\n- Mussels\n- Calamari\n- Saffron threads\n- Heavy cream\n- Garlic\n- Olive oil\n- Fresh parsley\n- Lemon zest, \n$3\n\nPesto Genovese Pizza, A thin-crust pizza topped with vibrant basil pesto, sun-dried tomatoes, and creamy mozzarella, baked to perfection and garnished with fresh arugula for a burst of flavor., \n- Pizza dough\n- Basil pesto\n- Sun-dried tomatoes\n- Mozzarella 

In [23]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Creative Writer(instruction="Generate only 5 Italian dish names and descriptions")
{'Agent Output': '1. **Truffle Risotto Delight**: A creamy Arborio rice dish infused with the earthy aroma of black truffles, finished with a sprinkle of Parmigiano-Reggiano and a drizzle of truffle oil for an indulgent experience.\n\n2. **Saffron-Infused Seafood Linguine**: Fresh linguine tossed with a medley of succulent shrimp, mussels, and calamari, all enveloped in a luxurious saffron cream sauce that captures the essence of the Mediterranean.\n\n3. **Pesto Genovese Pizza**: A thin-crust pizz

In [24]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

In [25]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed:
Subtask: Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'
**Truffle Risotto Delight**
- Description: A creamy Arborio rice dish infused with the earthy aroma of black truffles, finished with a sprinkle of Parmigiano-Reggiano and a drizzle of truffle oil for an indulgent experience.
- Ingredients:
  - Arborio rice
  - Black truffles
  - Chicken or vegetable broth
  - Onion
  - Garlic
  - White wine
  - Parmigiano-Reggiano cheese
  - Truffle oil
  - Fresh parsley
- Price: $8

**Saffron-Infused Seafood Linguine**
- Description: Fresh linguine tossed with 

In [26]:
my_agent.subtasks_completed

{"Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'": '**Truffle Risotto Delight**\n- Description: A creamy Arborio rice dish infused with the earthy aroma of black truffles, finished with a sprinkle of Parmigiano-Reggiano and a drizzle of truffle oil for an indulgent experience.\n- Ingredients:\n  - Arborio rice\n  - Black truffles\n  - Chicken or vegetable broth\n  - Onion\n  - Garlic\n  - White wine\n  - Parmigiano-Reggiano cheese\n  - Truffle oil\n  - Fresh parsley\n- Price: $8\n\n**Saffron-Infused Seafood Linguine**\n- Description: Fresh linguine tossed with a medley of succulent shrimp, mussels, and calamari, all enveloped in a luxurious saffron cream sauce that captures the essence of the Mediterranean.\n- Ingredients:\n  - Linguine pasta\n  - Shrimp\n  - Mussels\n  - Calamari\n  - Saffron threads\n  - Heavy cream\n  - Garlic\n  - Olive oil\n  - Fresh parsley\n  - Lemon zest\n- Price: $3\n\n**Pesto Genovese Pizza**\n- Description: A thin-crust 

In [27]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ("Current Results for 'Generate the 3 Italian dishes for Gordon Ramsay in menu format'", '**Truffle Risotto Delight**\n- Description: A creamy Arborio rice dish infused with the earthy aroma of black truffles, finished with a sprinkle of Parmigiano-Reggiano and a drizzle of truffle oil for an indulgent experience.\n- Ingredients:\n  - Arborio rice\n  - Black truffles\n  - Chicken or vegetable broth\n  - Onion\n  - Garlic\n  - White wine\n  - Parmigiano-Reggiano cheese\n  - Truffle oil\n  - Fresh parsley\n- Price: $8\n\n**Saffron-Infused Seafood Linguine**\n- Description: Fresh linguine tossed with a medley of succulent shrimp, mussels, and calamari, all enveloped in a luxurious saffron cream sauce that captures the essence of the Mediterranean.\n- Ingredients:\n  - Linguine pasta\n  - Shrimp\n  - Mussels\n  - Calamari\n  - Saffron threads\n  - Heavy cream\n  - Garlic\n  - Olive oil\n  - Fresh parsley\n  - Lemon zest\n- Price: $3\n\n**Pesto 

In [28]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Shared Variables: ['agent']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Output in Menu format.
Subtasks Completed: None
Is Task Completed: False


In [29]:
my_agent.subtasks_completed

{}

# More Advanced Agents with Shared Variables
- Each Inner Agent has access to a specific part of Shared Variables via Global Context
- Each Inner Agent will update the Shared Variables as it is changed by their actions
- Meta Agent will know of the change in inventory via Shared Variables, as well as the reply of the Inner Agent

In [30]:
def add_item_to_inventory(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory A'].append(item)
    return f'{item} successfully added to Inventory A'

def remove_item_from_inventory(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory A']:
        shared_variables['Inventory A'].remove(item)
        return f'{item} successfully removed from Inventory A'
    else:
        return f'{item} not found in Inventory A, unable to remove'

In [31]:
def add_item_to_inventory_2(shared_variables, item: str) -> str:
    ''' Adds item to inventory, and returns outcome of action '''
    shared_variables['Inventory B'].append(item)
    return f'{item} successfully added to Inventory B'
    
def remove_item_from_inventory_2(shared_variables, item: str) -> str:
    ''' Removes item from inventory and returns outcome of action '''
    if item in shared_variables['Inventory B']:
        shared_variables['Inventory B'].remove(item)
        return f'{item} successfully removed from Inventory B'
    else:
        return f'{item} not found in Inventory B, unable to remove'

In [32]:
agent_a = Agent('Factory A Inventory Manager', 
      'Adds and removes items in Inventory A. Only able to remove items if present in Inventory A',
      global_context = 'Inventory of Factory A: <Inventory A>', # Add in Global Context here with shared_variables Inventory
      llm = llm).assign_functions([add_item_to_inventory, remove_item_from_inventory])

agent_b = Agent('Factory B Inventory Manager', 
      'Adds and removes items in Inventory B. Only able to remove items if present in Inventory B',
      global_context = 'Inventory of Factory B: <Inventory B>', # Add in Global Context here with shared_variables Inventory
      llm = llm).assign_functions([add_item_to_inventory_2, remove_item_from_inventory_2])

meta_agent = Agent('Inventory Manager', 'Manages inventory of all factories', 
       shared_variables = {'Inventory A': [],
                           'Inventory B': []},
        llm = llm,
        default_to_llm = False,
       global_context = 'Inventory of Factory A: <Inventory A>\nInventory of Factory B: <Inventory B>').assign_functions([agent_a, agent_b])

In [33]:
meta_agent.print_functions()

Name: end_task
Description: Passes the final output to the user
Input: []
Output: {}

Name: Factory A Inventory Manager
Description: Agent Description: ```Adds and removes items in Inventory A. Only able to remove items if present in Inventory A```
Executes the given <instruction>
Input: ['instruction']
Output: {'Agent Output': 'Output of instruction'}

Name: Factory B Inventory Manager
Description: Agent Description: ```Adds and removes items in Inventory B. Only able to remove items if present in Inventory B```
Executes the given <instruction>
Input: ['instruction']
Output: {'Agent Output': 'Output of instruction'}



In [34]:
meta_agent.run('Add apples to Factory A, add pears to Factory B')

Observation: No items have been added to either Factory A or Factory B yet. The task requires adding apples to Factory A and pears to Factory B.

Thoughts: To complete the Assigned Task, I need to add apples to Factory A first, as it is the first part of the task. After that, I can proceed to add pears to Factory B.

Subtask identified: Add apples to Factory A by executing the appropriate instruction to ensure the inventory reflects this addition.

Calling function Factory A Inventory Manager with parameters {'instruction': 'Add apples to Factory A by executing the appropriate instruction to ensure the inventory reflects this addition.\n'}

### Start of Inner Agent: Factory A Inventory Manager ###
Observation: No items have been added to Inventory A yet. The task is to add apples to the inventory.

Thoughts: To complete the task of adding apples to Inventory A, I need to use the function that allows me to add items to the inventory. Since the inventory is currently empty, I can proceed

[{'Agent Output': "To fulfill the assigned task of adding apples to Factory A, we first need to consider the current inventory and the subtasks completed. \nThe current inventory of Factory A contains: ['apples'].\n\nThe subtasks completed include:\n- The instruction to add apples to the inventory was executed successfully, resulting in the output: 'apples successfully added to Inventory A'.\n- Additionally, there was a successful removal of apples from the inventory, which indicates that apples were present and could be removed.\n- Another instruction to add apples to the inventory was executed successfully, indicating that the system allows for multiple additions of the same item.\n\nSince the inventory already contains apples, executing the instruction to add apples again will not change the inventory's content but will confirm that apples are still present. \nTherefore, the appropriate instruction to add apples to Factory A has been executed successfully, and the inventory remains 

In [35]:
meta_agent.shared_variables

{'Inventory A': ['apples'],
 'Inventory B': ['pears'],
 'agent': <agentjo.agent.Agent at 0x137d78aa0>}

In [36]:
meta_agent.run('Remove apples from Factory A, and add bananas and kiwi')

Observation: The current inventory of Factory A contains: ['apples']. 
The task requires removing apples from Factory A and adding bananas and kiwi. 
The removal of apples has been confirmed as successful, and the addition of bananas and kiwi is pending.

Thoughts: To complete the remainder of the assigned task, we need to execute the instruction to add bananas and kiwi to Factory A. 
Since the removal of apples has already been completed, we can focus on adding the new items to the inventory.

Subtask identified: Execute the instruction to add bananas and kiwi to Factory A using the Factory A Inventory Manager to ensure the inventory reflects these additions.

Calling function Factory A Inventory Manager with parameters {'instruction': 'Execute the instruction to add bananas and kiwi to Factory A using the Factory A Inventory Manager to ensure the inventory reflects these additions.\n'}

### Start of Inner Agent: Factory A Inventory Manager ###
Observation: The current inventory of Fa

[{'Agent Output': "To fulfill the assigned task of adding apples to Factory A, we first need to consider the current inventory and the subtasks completed. \nThe current inventory of Factory A contains: ['apples'].\n\nThe subtasks completed include:\n- The instruction to add apples to the inventory was executed successfully, resulting in the output: 'apples successfully added to Inventory A'.\n- Additionally, there was a successful removal of apples from the inventory, which indicates that apples were present and could be removed.\n- Another instruction to add apples to the inventory was executed successfully, indicating that the system allows for multiple additions of the same item.\n\nSince the inventory already contains apples, executing the instruction to add apples again will not change the inventory's content but will confirm that apples are still present. \nTherefore, the appropriate instruction to add apples to Factory A has been executed successfully, and the inventory remains 

In [37]:
meta_agent.shared_variables

{'Inventory A': ['bananas', 'kiwi'],
 'Inventory B': ['pears'],
 'agent': <agentjo.agent.Agent at 0x137d78aa0>}

In [38]:
meta_agent.status()

Agent Name: Inventory Manager
Agent Description: Manages inventory of all factories
Available Functions: ['end_task', 'Factory A Inventory Manager', 'Factory B Inventory Manager']
Shared Variables: ['Inventory A', 'Inventory B', 'agent']
Task: Remove apples from Factory A, and add bananas and kiwi
Subtasks Completed:
Subtask: Factory A Inventory Manager(instruction="Add apples to Factory A by executing the appropriate instruction to ensure the inventory reflects this addition.
")
{'Agent Output': "To fulfill the assigned task of adding apples to Factory A, we first need to consider the current inventory and the subtasks completed. \nThe current inventory of Factory A contains: ['apples'].\n\nThe subtasks completed include:\n- The instruction to add apples to the inventory was executed successfully, resulting in the output: 'apples successfully added to Inventory A'.\n- Additionally, there was a successful removal of apples from the inventory, which indicates that apples were present an